### Checklist

* Re-name header to ROI
* The script and the csv file has to be in the same folder, if not modify path
* Enter correct values for baseline and treatments
* For all cells with plot_ manually select the ROIs 
* Cross-check logic in cells where data frames are being merged
* Change labels appropriately in summary table
* Name the export sheets appropriately


In [1]:
import pandas as pd
import openpyxl 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
#import csv
raw = "" #input correct path

In [3]:
#### Defining Normalization function

def Normalization(baseline):
    raw_df = pd.read_excel(raw)
    average = raw_df.loc[raw_df["Time (sec)"] <= baseline]
    average = average.set_index('Time (sec)').mean()

# Setting index to help with calculation
    raw_df = raw_df.set_index("Time (sec)")

# Normalizing
    normalization = raw_df/average
    normalization = normalization.reset_index()
    return normalization

#### Defining function for calculating responders
'''input the start and end time for each treatment'''

def Responders(start, end):
    
# Determining the value at the start time of drug addition and determining the cut-off for response
    start_value = normalization.loc[normalization["Time (sec)"] == start]
    start_value = start_value.set_index("Time (sec)")
    cut_off = start_value + 0.2
    
#Selecting the duration of treatment
    duration = normalization.loc[(normalization["Time (sec)"] >= start) & (normalization["Time (sec)"] <= end)]
    duration = duration.set_index("Time (sec)")
    duration_max = duration.max()
    
#Determining responders
    responders = duration_max > cut_off
    responders = responders.transpose().reset_index()
    responders.columns=["cells", "response"]
    responders = responders.loc[responders["response"] == True]
    return responders


In [14]:
#Call the functions
normalization = Normalization(#insert baseline value) 
normalization.head(2)

,Time (sec),ROI 1,ROI 2,ROI 3,ROI 4,ROI 5,ROI 6,ROI 7,ROI 8,ROI 9,...,ROI 51,ROI 52,ROI 53,ROI 54,ROI 55,ROI 56,ROI 57,ROI 58,ROI 59,ROI 60
0,0.58,0.972484,0.977364,0.971757,0.962826,0.957783,0.976978,0.968941,0.967598,0.974217,...,0.985706,0.967781,0.974891,0.994969,0.965882,0.946987,0.970416,0.991350,0.963023,0.97517
1,3.59,0.976101,0.969516,0.976093,0.975765,0.955541,0.968165,0.978052,0.970165,0.977147,...,0.980226,0.984814,0.970063,0.989208,0.986718,0.950197,0.977047,0.982816,0.973881,0.98401


In [10]:
#Calculate responders for all treatments by defining t2_responders, t3_responders etc

t1_responders = Responders(#insert start , #insert end time )
t1_responders

,cells,response
0,ROI 1,True
24,ROI 25,True


In [ ]:
# select the correct labels from the output of the previous cells to call in the correct ROIs
plot_t1 = normalization[["Time (sec)", \
                         "ROI ", "ROI "]]

In [ ]:
# Cells that responded to any beta-alanine treatment : Choose all treatments where beta-alanine was added sequential merging
# Change labels
any_ba = pd.merge(t1_responders, t2_responders, on="cells", how="outer")
any_ba = pd.merge(any_ba, t3_responders, on="cells", how="outer")

#Rename accurately
any_ba = any_ba.rename(columns={"response_x": "Beta-alanine (1um)",
                                "response_y": "Beta-alanine (5um)",
                                "response": "Beta-alanine (10um)"})
any_ba

In [ ]:
plot_any_ba = normalization[["Time (sec)", 
                         "ROI ", "ROI "]]

In [ ]:
#cells that responded to all beta-alanine treatment: Choose all treatments where beta-alanine was added
all_ba = pd.merge(t1_responders, t2_responders, on="cells", how="inner")
all_ba = pd.merge(all_ba, t3_responders, on="cells", how="inner")

all_ba = all_ba.rename(columns={"response_x": "Beta-alanine (1um)",
                                "response_y": "Beta-alanine (5um)",
                                "response": "Beta-alanine (10um)"})
all_ba

In [ ]:
plot_all_ba = normalization[["Time (sec)"]]

In [ ]:
#cells that responded to both capsaicin and any beta-alanine
ba_cap = pd.merge(any_ba, t4_responders, on="cells", how="inner")
ba_cap = ba_cap.rename(columns={"response": "Capsaicin"})
ba_cap

In [ ]:
plot_ba_cap = normalization[["Time (sec)", 
                         "ROI ", "ROI "]]

In [ ]:
# cells that did responded to any treatment
any_treatment = pd.merge(any_ba, t4_responders, how="outer")
any_treatment = any_treatment.rename(columns={"response": "Capsaicin"})
any_treatment = pd.merge(any_treatment, t5_responders, how="outer")
any_treatment = any_treatment.rename(columns={"response": "HK"})
any_treatment

In [ ]:
#Determining % of reponders
per_t1_responders = round(t1_responders["cells"].count()/any_treatment["cells"].count()*100, 2)
per_t2_responders = round(t2_responders["cells"].count()/any_treatment["cells"].count()*100, 2)
per_t3_responders = round(t3_responders["cells"].count()/any_treatment["cells"].count()*100, 2)
per_t4_responders = round(t4_responders["cells"].count()/any_treatment["cells"].count()*100, 2)
per_t5_responders = round(t5_responders["cells"].count()/any_treatment["cells"].count()*100, 2)

per_any_ba = round(any_ba["cells"].count()/any_treatment["cells"].count()*100, 2)
per_all_ba = round(all_ba["cells"].count()/any_treatment["cells"].count()*100, 2)
per_ba_cap = round(ba_cap["cells"].count()/any_treatment["cells"].count()*100, 2)


In [ ]:
treatment_summary = pd.DataFrame({"Beta alanine (1um)": [per_t1_responders],
                      "Beta alanine (5um)": [per_t2_responders],
                      "Beta alanine (10um)": [per_t3_responders],
                      "Capsaicin (10 um)": [per_t4_responders],
                       "HK": [per_t5_responders],
                       "Any Beta alanine": [per_any_ba],
                       "All Beta alanine": [per_all_ba],
                       "Beta alanine and capsaicin": [per_ba_cap]})

treatment_summary


In [ ]:
# Exporting files

with pd.ExcelWriter('output.xlsx') as writer:  
    plot_t1.to_excel(writer, sheet_name='t1', index=False)
    plot_t2.to_excel(writer, sheet_name='t2', index=False)
    plot_t3.to_excel(writer, sheet_name='t3', index=False)
    plot_t4.to_excel(writer, sheet_name='t4', index=False)
    plot_t5.to_excel(writer, sheet_name='t5', index=False)
    plot_any_ba.to_excel(writer, sheet_name='any_ba', index=False)
    plot_all_ba.to_excel(writer, sheet_name='all_ba', index=False)
    plot_ba_cap.to_excel(writer, sheet_name='ba_cap', index=False)
    treatment_summary.to_excel(writer, sheet_name='summary.csv', index=False)
    